In [5]:
""" pip install beautifulsoup4
pip install emoji """

' pip install beautifulsoup4\npip install emoji '

In [6]:
# pip install gspread google-auth

In [7]:
# Import Libraries
from bs4 import BeautifulSoup
import requests
import gspread
from google.auth import exceptions
from google.auth.transport.requests import Request
from google.oauth2.service_account import Credentials
import pandas as pd
import time

import pygsheets

In [8]:
#authorization
gc = pygsheets.authorize(service_file='/Users/asessums/Desktop/nomad-list-datascraper/credentials.json')

In [9]:
# Function to extract data from a given URL
def extract_data(url, city_name):
    print(f"Extracting data for {city_name} from {url}")
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code != 200:
        print(f"Failed to retrieve data for {city_name}. Status code: {response.status_code}")
        return []

    html_content = response.content
    soup = BeautifulSoup(html_content, 'html.parser')

    # Extract details from the <tr> elements
    details_rows = soup.select('table.details tr')
    city_data = []

    # Extract Continent and Country
    continent = soup.select_one('td.key:contains("Continent") + td.value a')['title']
    country = soup.select_one('td.key:contains("Country") + td.value a')['title']

    for row in details_rows:
        key_element = row.select_one('.key')
        value_element = row.select_one('.value')

        if not key_element or not value_element:
            print(f"Skipping row for {city_name} as key or value element is missing.")
            continue

        # Extract label, data value, and div value
        label = key_element.text.strip()
        data_value = None
        quality_rating = None

        if value_element.select_one('.filling'):
            # Check if div_value is a number
            try:
                data_value = float(value_element.select_one('.filling').text.strip())
            except ValueError:
                quality_rating = value_element.select_one('.filling').text.strip()

        elif value_element.select_one('.rating'):
            data_value = float(value_element.select_one('.rating')['data-value'])
            quality_rating = value_element.select_one('.filling').text.strip()
        else:
            quality_rating = value_element.text.strip()

        # Append to the city_data list
        city_data.append([city_name, label, quality_rating, data_value, continent, country])

    return city_data

# List of cities with their URLs
cities = [
    {"name": "Berlin", "url": "https://nomadlist.com/berlin"},
    {"name": "Stockholm", "url": "https://nomadlist.com/stockholm"},
    # {"name": "Barcelona", "url": "https://nomadlist.com/barcelona"},
    # {"name": "Delhi", "url": "https://nomadlist.com/delhi"},
    # {"name": "Tokyo", "url": "https://nomadlist.com/tokyo"},
    # {"name": "Lagos", "url": "https://nomadlist.com/lagos"},
    # {"name": "Nairobi", "url": "https://nomadlist.com/nairobi"},
    # {"name": "Cape Town", "url": "https://nomadlist.com/cape-town"},
    # {"name": "Lisboa", "url": "https://nomadlist.com/lisbon"},
    # ... (add more cities here)
]

# Iterate over cities
all_data = []

for city in cities:
    city_name = city["name"]
    city_url = city["url"]

    # Extract data for the current city
    city_data = extract_data(city_url, city_name)

    # Extend all_data with the individual data points for each city
    all_data.extend(city_data)

    # Add a delay of 20 seconds between requests
    time.sleep(20)

# Create Pandas DataFrame
headers = ["City", "Label", "Quality Rating", "Data Value", "Continent", "Country"]
df = pd.DataFrame(all_data, columns=headers)

df.tail(10)

# Create Pandas DataFrame with adjusted column order
headers = ["City", "Label", "Quality Rating", "Data Value", "Continent", "Country"]
df = pd.DataFrame(all_data, columns=headers)

df.tail(10)

Extracting data for Berlin from https://nomadlist.com/berlin


/Users/asessums/miniforge3/envs/stenv/lib/python3.10/site-packages/soupsieve/css_parser.py:876: FutureWarning: The pseudo class ':contains' is deprecated, ':-soup-contains' should be used moving forward.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
# Move dataframe into a Google sheet
spreadsheet = gc.open('Digital Nomad List')

#select the first sheet 
worksheet = spreadsheet[0]

#update the first sheet with df, starting at cell B2. 
worksheet.set_dataframe(df,(1,1))